In [35]:
import pandas as pd

In [ ]:
txp_file = pd.read_csv("/home/jovyan/ifbdata/spatial_cell_id/Reference/txp2gene/txp2gene_reduced.tsv", sep="\t")
value = txp_file.iloc[:,1]
# print(len(value))
value = value.append(pd.Series(["ENSG00000186092.7"]), ignore_index=True) #manually adding gene left out as column header 
# print(len(value))
protein_coding_genes = set(value)
# print(protein_coding_genes)
print(len(protein_coding_genes))

output_file = "/home/jovyan/ifbdata/spatial_cell_id/Reference/txp2gene/protein_coding_genes_versions.txt"
with open(output_file, "w") as file:
    for gene in protein_coding_genes:
        file.write(f"{gene}\n")

# used find and replace and regex to remove the version suffix for symbol conversion

In [8]:
# databases used for gene_ID to gene_symbol conversion:

# https://biotools.fr/human/ensembl_symbol_converter
# https://www.syngoportal.org/convert

gene_id = "/home/jovyan/ifbdata/spatial_cell_id/Reference/txp2gene/protein_coding_genes.txt"
gene_symbol = "/home/jovyan/ifbdata/spatial_cell_id/Reference/txp2gene/gene_symbols.txt"
output = "/home/jovyan/ifbdata/spatial_cell_id/Kush/alignment/demo_matrix/genes_demo.tsv"

with open(gene_id, "r") as file1, open(gene_symbol, "r") as file2:
    values1 = file1.readlines()
    values2 = file2.readlines()

if len(values1) != len(values2):
    raise ValueError("The two files must have the same number of lines.")

with open(output, "w") as output_file:
    output_file.write("Gene_ID\tGene_symbol\n")
    
    for val1, val2 in zip(values1, values2):
        output_file.write(f"{val1.strip()}\t{val2.strip()}\n")

print(f"TSV file created at {output}")

# got the indeces of empty rows in gene_symbols.txt for index in index_gene_symbol.txt
# genes.tsv --> genes_demo.tsv

TSV file created at /home/jovyan/ifbdata/spatial_cell_id/Kush/alignment/demo_matrix/genes_demo.tsv


In [26]:
# extracted the gene_IDs for corresposnding missing gene symbols 

df = pd.read_csv("/home/jovyan/ifbdata/spatial_cell_id/Reference/txp2gene/genes.tsv", sep="\t")

with open('/home/jovyan/ifbdata/spatial_cell_id/Kush/alignment/demo_matrix/index_gene_symbol.txt', 'r') as file:
    index = [int(line.strip()) for line in file]

output_df = pd.DataFrame({
    'Original_Index': index,  # original indices
    'Gene_ID': values
})

# values = df.iloc[index - 1]["Gene_ID"]
output_df.to_csv("/home/jovyan/ifbdata/spatial_cell_id/Kush/alignment/demo_matrix/missing_symbol.tsv", sep='\t', header=True, index=False)

# created header for reference, to be removed at last step

In [29]:
# extracted symbols for as much available IDs from other Database
# updating the genes.tsv file

# Load the TSV and Excel files
file1 = pd.read_csv("/home/jovyan/ifbdata/spatial_cell_id/Kush/alignment/demo_matrix/genes.tsv", sep="\t")
file2 = pd.read_excel("/home/jovyan/ifbdata/spatial_cell_id/Kush/alignment/demo_matrix/idmap.xlsx")

# Ensure column names are consistent for easy merging
file1.columns = file1.columns.str.strip()
file2.columns = file2.columns.str.strip()

# Create a dictionary from file2 with gene_ID as key and gene_symbol as value
file2_dict = dict(zip(file2['query'], file2['symbol']))

# Update file1
def update_gene_symbol(row):
    if pd.isna(row['Gene_symbol']):
        if row['Gene_ID'] in file2_dict:
            return file2_dict[row['Gene_ID']] if not pd.isna(file2_dict[row['Gene_ID']]) else f"UND_{row['Gene_ID']}"
        else:
            return f"UND_{row['Gene_ID']}"
    return row['Gene_symbol']

file1['Gene_symbol'] = file1.apply(update_gene_symbol, axis=1)

# Save the updated DataFrame to a new TSV file
file1.to_csv("/home/jovyan/ifbdata/spatial_cell_id/Kush/alignment/demo_matrix/updated_gene_symbol.tsv", sep="\t", index=False)
print("done")

done


In [32]:
# checking the order of the genes.tsv file after update

with open("/home/jovyan/ifbdata/spatial_cell_id/Reference/txp2gene/protein_coding_genes.txt", 'r') as file:
    values1 = [line.strip() for line in file]

tsv_data = pd.read_csv("/home/jovyan/ifbdata/spatial_cell_id/Kush/alignment/demo_matrix/updated_gene_symbol.tsv", sep="\t")
values2 = tsv_data["Gene_ID"].tolist()

if values1 == values2:
    print("sequence matches")
else:
    print("sequence does not match")

sequence matches


In [34]:
# removing the header created in initial step

file_path = "/home/jovyan/ifbdata/spatial_cell_id/Reference/txp2gene/genes.tsv"

df = pd.read_csv(file_path, sep="\t", header=0)

df.to_csv(file_path, sep="\t", index=False, header=False)